## Problem Set 2
Daniela Santos Cárdenas, Ana Brás Monteiro

In [10]:
# Importing the necessary libraries
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np
from linearmodels.iv import IV2SLS
from statsmodels.sandbox.regression.gmm import GMM
from statsmodels.sandbox.regression import gmm
from scipy.linalg import block_diag

In [2]:
# Import data 

colnames=['mkt_id', 'prod_id', 'prod_share', 'prod_att1', 'prod_att2', 'prod_att3', 'price', 'shifter1', 'shifter2', 'shifter3', 'group'] 

df = pd.read_csv('../data/Data.csv', names=colnames, header=None)

In [3]:
df.describe()

,mkt_id,prod_id,prod_share,prod_att1,prod_att2,prod_att3,price,shifter1,shifter2,shifter3,group
count,970.000000,970.000000,9.700000e+02,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000,970.000000
mean,26.167010,10.430928,1.912517e-02,-0.030330,-0.004903,-0.057823,5.577779,-0.029543,-0.012141,-0.044407,2.010309
std,14.485213,5.990177,6.025534e-02,0.995995,0.999779,1.016903,1.371331,1.007472,0.989332,0.977154,0.818956
min,1.000000,1.000000,3.019000e-07,-2.985200,-3.104500,-4.167300,1.328500,-2.850200,-4.096800,-3.251200,1.000000
25%,14.000000,5.000000,9.976725e-05,-0.706257,-0.682220,-0.706403,4.708150,-0.710245,-0.704515,-0.700940,1.000000
50%,26.500000,10.000000,7.848100e-04,-0.064417,0.042951,-0.068317,5.527950,-0.055330,0.017694,-0.001928,2.000000
75%,39.000000,15.000000,5.950150e-03,0.670627,0.681762,0.620657,6.507125,0.688550,0.695118,0.590225,3.000000
max,50.000000,25.000000,5.582900e-01,2.811000,3.690400,3.231500,10.000000,3.275500,3.681100,3.272600,3.000000


## 2.1 Logit Demand

**Question a)**

Estimate an aggregate Logit model using OLS based on the following utility function that individual i derives from buying product j in market n: 

$$u_{ijn} = \alpha p_{jn} + x_{jn} \beta + \xi_{jn} + \varepsilon_{ijn}$$

Dependent variable: $\delta_{jn} = log(\frac{s_{jn}}{s_{0n}})$

In [4]:
# Creating market shares 
df['mkt_share'] = df.groupby('mkt_id')['prod_share'].transform('sum')

# Share of the outside good 
df['mkt_share_out'] = 1 - df['mkt_share']

# Calculate log of ratio 
df['utility'] = np.log(df['prod_share']/df['mkt_share_out'])


In [5]:
# OLS
ols_logit = smf.ols('utility ~ 1 + price + prod_att1 + prod_att2 + prod_att3', data=df).fit()
ols_logit.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                utility   R-squared:                       0.843
Model:                            OLS   Adj. R-squared:                  0.842
Method:                 Least Squares   F-statistic:                     1294.
Date:                Wed, 30 Nov 2022   Prob (F-statistic):               0.00
Time:                        13:07:46   Log-Likelihood:                -1491.9
No. Observations:                 970   AIC:                             2994.
Df Residuals:                     965   BIC:                             3018.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.9496      0.163      5.818      0.000       0.629       1.270
price         -1.3385      0.028    -47.054      0.000      -1.394      -1.283
prod_att1      2.4160      0.045     53.906      0.000       2.328       2.504
prod_att2      0.5124      0.045     11.319      0.000       0.424       0.601
prod_att3      0.3696      0.043      8.689      0.000       0.286       0.453
==============================================================================
Omnibus:                        2.031   Durbin-Watson:                   1.837
Prob(Omnibus):                  0.362   Jarque-Bera (JB):                2.082
Skew:                           0.088   Prob(JB):                        0.353
Kurtosis:                       2.857   Cond. No.                         26.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**Question b)**

Estimate the same Logit model using Instrumental Variables (IV). Use the cost shifters as instruments, providing the results also for the 1st stage. How do your results change compared to the OLS case? Provide an intuition for the endogeneity bias. Calculate the mean across markets of own and cross price elasticities.

In [6]:
iv_logit = IV2SLS.from_formula('utility ~ 1 + [price ~ shifter1 + shifter2 + shifter3] + prod_att1 + prod_att2 + prod_att3', data = df).fit()
iv_logit.summary


<class 'linearmodels.compat.statsmodels.Summary'>
"""
                          IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:                utility   R-squared:                      0.8400
Estimator:                    IV-2SLS   Adj. R-squared:                 0.8393
No. Observations:                 970   F-statistic:                    3905.7
Date:                Wed, Nov 30 2022   P-value (F-stat)                0.0000
Time:                        13:07:49   Distribution:                  chi2(4)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      1.6129     0.2304     7.0003     0.0000      1.1613      2.0644
prod_att1      2.4668     0.0478     51.598     0.0000      2.3731      2.5605
prod_att2      0.5616     0.0491     11.435     0.0000      0.4653      0.6579
prod_att3      0.3725     0.0444     8.3935     0.0000      0.2855      0.4595
price         -1.4571     0.0417    -34.962     0.0000     -1.5388     -1.3754
==============================================================================

Endogenous: price
Instruments: shifter1, shifter2, shifter3
Robust Covariance (Heteroskedastic)
Debiased: False
"""

**Elasticities** 

Own-price elasticities: $\epsilon_{jj} = \alpha p_{jn} (1-s_{jn})$

Cross-price elasticities: $\epsilon_{jk} = - \alpha p_{kn} s_{kn}$

In [7]:
# Own price elasticities 
df['e_jj'] = iv_logit.params['price'] * df['price'] * (1 - df['prod_share'])
# Mean across markets 
np.mean(df.e_jj)

-7.993652493251175

In [8]:
# Cross-price elasticities 
df['e_jk'] = - iv_logit.params['price'] * df['price'] * df['prod_share']
# Mean across markets
np.mean(df.e_jk) 

# I tried weighting the means by market share but it doesn't change much. Not sure what's ideal. 

0.13366860095027963

**Question c)**

You will need to construct a GMM objective function with both demand and supply moments, with the price coefficient α entering in both moments (cross-equation restriction). How do your results change compared to the case with just IV?


In [ ]:
## Resources that I used to learn about this: 
# Carole's help 
# Info on classes and OOP: https://realpython.com/python3-object-oriented-programming/
# GMM estimator: https://www.statsmodels.org/stable/generated/statsmodels.sandbox.regression.gmm.GMM.html#statsmodels.sandbox.regression.gmm.GMM

In [11]:
# Define exog, endog and instruments 

instrument_df = df[["shifter1", "shifter2","shifter3"]]  # instruments
exog_df = df[["price", "prod_share", "prod_att1", "prod_att2","prod_att3"]]  # explanatory variables 
endog_df = df[["utility", "price"]] # dependent variable
endog, exog, instrument  = map(np.asarray, [endog_df, exog_df, instrument_df]) # Creates matrices


class GMMdemand(GMM): 
    
    def momcond(self, params): # This is where we define moment conditions 
        alpha, beta1, beta2, beta3, gamma1, gamma2, gamma3, k1, k2 = params # define params  
        beta = np.array([beta1,beta2,beta3])
        gamma = np.array([gamma1,gamma2,gamma3])

        x = self.exog # endog, exog, instrument are inputs in the parent class 
        z = self.instrument
        y = self.endog 

        n_obs = z.shape[0] # Number of observations (shape[0] returns the row dimensions of the array)
        exogenous_x = x[:, 2:] # Product characteristics 
        share = x[:, 1] # Market share
        price = x[:, 0] # Price
        log_share = y[:, 0] # Log market share


        m1 = np.array(log_share - k1 * np.ones(n_obs) - alpha * price - beta.dot(exogenous_x.T)).reshape(1,n_obs) # Demand side 
        m2 = np.array(price - k2 * np.ones(n_obs) - gamma.dot(z.T) - 1/(alpha*(1-share))).reshape(1,n_obs)  # Supply side 
        vector_instruments_m1 = np.concatenate((np.ones(n_obs).reshape(1, n_obs), exogenous_x.T, z.T), axis = 0) # Create the instrument matrix so we can multiply our moment conditions (Z'e)
        vector_instruments_m2 = np.concatenate((np.ones(n_obs).reshape(1, n_obs), z.T), axis = 0) # Create the instrument matrix so we can multiply our moment conditions (Z'e)
        mat_instruments =  block_diag(vector_instruments_m1, vector_instruments_m2) 
        sevenm1 = np.concatenate((m1, m1,m1, m1, m1, m1, m1), axis = 0) 
        fourm2 = np.concatenate((m2, m2,m2, m2), axis = 0)
        errors = block_diag(sevenm1, fourm2) # (11, 1940) (fill the off-diagonal blocks with zeros)
        moments = (mat_instruments*errors).T # Create the 11 moment conditions (7 for demand and 4 for supply): Z'e which will be minimized by the command GMM

        return moments


model1 = GMMdemand(endog, exog, instrument, k_moms = 11, k_params=9) 
b0 = np.array([-1.5, 1, 1, 1, 1, 1, 1, 1, 1]) # Initial guess
res1 = model1.fit(b0, maxiter=100, optim_method='bfgs', wargs=dict(centered=False)) # fit is a method of the GMM class but model1 is both an object of the child class GMMdemand (which assigns the momcond that the parent class GMM requires) and an object of the parent class GMM (which has the method fit)
xnames = ["alpha", "beta1", "beta2", "beta3", "gamma1", "gamma2", "gamma3", "k1", "k2"]
print(res1.summary(xname = xnames))


Optimization terminated successfully.
         Current function value: 0.000107
         Iterations: 23
         Function evaluations: 24
         Gradient evaluations: 24
Optimization terminated successfully.
         Current function value: 0.000703
         Iterations: 14
         Function evaluations: 15
         Gradient evaluations: 15
Optimization terminated successfully.
         Current function value: 0.000703
         Iterations: 3
         Function evaluations: 6
         Gradient evaluations: 6
Optimization terminated successfully.
         Current function value: 0.000703
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1
                              GMMdemand Results                               
Dep. Variable:           ['y1', 'y2']   Hansen J:                        1.363
Model:                      GMMdemand   Prob (Hansen J):                 0.506
Method:                           GMM                                         
Da

In [12]:
res1.params[0] 

-1.4547489325113936

Results don't look so different from what we got from IV, suggestingn that the supply side doesn't seem to impact much the results. 

## 2.2 Nested Logit Demand

**Question a)**

Estimate a Nested Logit Model using IV based on the following: 

$$u_{ijn} = \alpha p_{jn} + x_{jn} \beta + \xi_{jn} + \zeta_{ign} + (1-\sigma)\varepsilon_{ijn}$$

Will estimate: 

$$log(\frac{s_{jn}}{s_{0n}}) = \alpha p_{jn} + x_{jn} \beta + \sigma log(\frac{s_{jn}}{s_{gn}}) + \xi_{jn}$$

Got help from https://pyblp.readthedocs.io/en/stable/_notebooks/tutorial/logit_nested.html

In [56]:
# Creating group shares 
df['group_share'] = df.groupby(['group', 'mkt_id'])['prod_share'].transform('sum')

# Creating ratio of product share to group share 
df['sjn_sgn'] = np.log(df['prod_share']/df['group_share'])


In [59]:
nest_logit = IV2SLS.from_formula('utility ~ 1 + [price ~ shifter1 + shifter2 + shifter3] + sjn_sgn + prod_att1 + prod_att2 + prod_att3', data=df).fit()
nest_logit.summary

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                          IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:                utility   R-squared:                      0.9057
Estimator:                    IV-2SLS   Adj. R-squared:                 0.9052
No. Observations:                 970   F-statistic:                    8292.7
Date:                Thu, Nov 24 2022   P-value (F-stat)                0.0000
Time:                        15:49:57   Distribution:                  chi2(5)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept     -0.0200     0.2560    -0.0783     0.9376     -0.5219      0.4818
prod_att1      2.1441     0.0441     48.616     0.0000      2.0577      2.2305
prod_att2      0.3213     0.0437     7.3596     0.0000      0.2357      0.4069
prod_att3      0.2620     0.0338     7.7564     0.0000      0.1958      0.3282
sjn_sgn        0.3908     0.0257     15.196     0.0000      0.3404      0.4413
price         -0.9155     0.0596    -15.356     0.0000     -1.0323     -0.7986
==============================================================================

Endogenous: price
Instruments: shifter1, shifter2, shifter3
Robust Covariance (Heteroskedastic)
Debiased: False
"""

**Own-price elasticities** 

Products not in a nest: $\epsilon_{jj} = \alpha p_{jn} (1-s_{jn})$

Products in a nest: $\epsilon_{jj} = \frac{\alpha p_{jn}}{1-\sigma} (1-\sigma s_{j|g,n} - (1-\sigma)s_{jn})$


In [60]:
alpha = nest_logit.params['price']
sigma = nest_logit.params['sjn_sgn']

In [63]:
# Products not in nest 
df['e_jj'] = alpha * df['price'] * (1 - df['prod_share'])
np.mean(df.e_jj)

-5.022401401654601

In [64]:
# Products in a nest 
df['e_jj_nest'] = (alpha * df['price'])/(1 - sigma) * (1 - sigma * (df['prod_share']/df['group_share']) - (1 - sigma) * df['prod_share'])
np.mean(df.e_jj_nest)

-7.883001635885313

**Cross-price elasticities** 

Products in different nests: $\epsilon_{jk} = - \alpha p_{kn} s_{kn}$

Products in the same nest: $\epsilon{jk} = - \frac{\alpha p_{kn}}{1 - \sigma}(\sigma s_{k|g,n} + (1 - \sigma)s_{kn})$

In [65]:
# Products in different nests 
df['e_jk'] = - alpha * df['price'] * df['prod_share']
np.mean(df.e_jk)

0.08398380706900704

In [66]:
# Products in the same nest 
df['e_jk_nest'] = - (alpha * df['price'])/(1 - sigma) * ( sigma * (df['prod_share']/df['group_share']) + (1 - sigma) * df['prod_share'])
np.mean(df.e_jk_nest)

0.4997047171412455